In [ ]:
PYMEPIX_IP = '127.0.0.1'
PYMEPIX_PORT = 5056
SERVE_PORT = 5010

In [ ]:
# Imports
import time
import numpy as np
import pandas as pd

import panel as pn
import param
#pn.extension('altair', 'tabulator')


from IPython.lib import backgroundjobs as bg

from pymepix.channel.client import Client
from pymepix.channel.channel_types import ChannelDataType, Commands

import holoviews as hv
from holoviews import opts
from holoviews import streams
import hvplot.pandas
from holoviews.streams import Pipe
from bokeh.plotting import show
hv.extension('bokeh')

"""
fontsize={'title': 10, 'labels': 10, 'xticks': 8, 'yticks': 8}
opts.defaults(
    opts.Scatter(width=1000, height=400, tools=["hover"], fontsize=fontsize, show_grid=True),# title=f"{git_info()}"),
    opts.Histogram(width=1000, height=400, tools=["hover"], fontsize=fontsize),# title=f"{git_info()}"),
    opts.Image(width=400, height=400, tools=["hover"], fontsize=fontsize),# title=f"{git_info()}"),
    opts.Curve(width=600, height=400, tools=["hover"], fontsize=fontsize, show_grid=True),# title=f"{git_info()}"),
    opts.Points(width=600, height=400, tools=["hover"], fontsize=fontsize, show_grid=True)#, title=f"{git_info()}"),
)"""


get_x_axis_from_bins = lambda x_bins: 0.5 * (x_bins[1:] + x_bins[:-1])

In [ ]:
client = Client((PYMEPIX_IP, PYMEPIX_PORT), None,)
data_queue = client.get_queue()

jobs = bg.BackgroundJobManager()

In [ ]:
pipe_roi_scan = Pipe(data=np.array([]))
pipe_tof = Pipe(data=np.array([]))
pipe_histo = Pipe(data=np.array([]))

DATA_FILTER = [ChannelDataType.TOF.value, ChannelDataType.COMMAND.value]
bins_VMI = (range(0, 256), range(0, 256))

Element_ROI = {
              "roi1": [[5.57, 5.67], np.zeros((255, 255))],
              "roi2": [[5.67, 5.77], np.zeros((255, 255))],
              "roi3": [[6.4, 6.5], np.zeros((255, 255))],
              "roi4": [[8.15, 8.25], np.zeros((255, 255))],
              }
ROI_colors = ['red', 'green', 'blue', 'orange']

H_VMI = np.zeros((255, 255), dtype=float)

bins_tof = np.linspace(0, 10, 500)
x_axe = bins_tof[:-1]
tof_accu = np.zeros((len(bins_tof)-1), dtype=float)

roi_scan_data = []

pipe_roi_scan.send(np.zeros((1,len(Element_ROI)+1)))
pipe_tof.send(H_VMI)
pipe_histo.send(tof_accu)

In [ ]:
def reset_button_callback(Event):
    global H_VMI, tof_accu, roi_scan_data
    H_VMI = np.zeros((255, 255), dtype=float)
    tof_accu = np.zeros((len(bins_tof)-1), dtype=float)
    roi_scan_data = []
    
    for roi_val in Element_ROI.values():
        roi_val[1] = np.zeros((255, 255), dtype=float)
    
    pipe_roi_scan.send(np.zeros((1,len(Element_ROI)+1)))
    pipe_tof.send(H_VMI)
    pipe_histo.send(tof_accu)

roi_select = pn.widgets.MultiChoice(
    name='ToF ROI', value=[list(Element_ROI.keys())[0]],
    options=list(Element_ROI.keys()),)

reset_button = pn.widgets.Button(name='Reset', button_type='primary')
reset_button.on_click(reset_button_callback)

In [ ]:
def main_loop(data_filter=DATA_FILTER):
    
    global H_VMI
    global tof_accu
    global x_axe
    global roi_scan_data
    global roi_select
    
    while True:
        data = data_queue.get()

        if data['type'] not in data_filter:
            continue
            
        if data['type'] == ChannelDataType.TOF.value:
            
            tof_data = data['data']
            mcs_tof = tof_data[3]*1e6
            H_VMI += np.histogram2d(tof_data[1], tof_data[2], bins=bins_VMI)[0]
            for roi_key, roi_val in Element_ROI.items():
                roi_indx = np.logical_and(mcs_tof>roi_val[0][0], mcs_tof<roi_val[0][1])
                roi_val[1] += np.histogram2d(tof_data[1][roi_indx], tof_data[2][roi_indx], bins=bins_VMI)[0]

            
            if roi_select.value == []:
                pipe_tof.send(H_VMI)
            else:
                roi_H_VMI = np.zeros((255, 255), dtype=float)
                for roi_key, roi_val in Element_ROI.items():
                    if roi_key in roi_select.value:
                        roi_H_VMI += roi_val[1]
                pipe_tof.send(roi_H_VMI)

            
            tof_accu += np.histogram(tof_data[3]*1e6, bins=bins_tof)[0]
            pipe_histo.send(tof_accu/tof_accu.max())
            
           
        if data['type'] == ChannelDataType.COMMAND.value:            
            if data['data'] == Commands.START_RECORD.value:
                H_VMI = np.zeros((255, 255), dtype=float)
                tof_accu = np.zeros((len(bins_tof)-1), dtype=float)
                for roi_val in Element_ROI.values():
                    roi_val[1] = np.zeros((255, 255), dtype=float)
                
            elif data['data'] == Commands.STOP_RECORD.value:
                scan_dataset = [len(roi_scan_data)]
                for roi in Element_ROI.values():
                    roi_indxs=np.logical_and(x_axe>roi[0][0], x_axe<roi[0][1])                    
                    scan_dataset.append(np.sum(tof_accu[roi_indxs]))
                    
                roi_scan_data.append(scan_dataset)                
                
                numpy_array = np.array(roi_scan_data)
                pipe_roi_scan.send(numpy_array)


                    
            

jobs.new(main_loop)
#main_loop()

In [ ]:
jobs.status()

In [ ]:
jobs.traceback()

In [ ]:
def make_roi_scan_plot(data):
    global roi_select
    list_of_curves = []
    roi_indxs = [roi_select.options.index(i) for i in roi_select.value]
    for indx in roi_indxs:
        list_of_curves.append(hv.Curve((data[:,0], data[:, indx+1])).opts(xlabel='scan num.', ylabel='amplitude, [au]', axiswise=True, width=600, height=450, show_grid=True, tools=['hover'], color=ROI_colors[indx])* hv.Scatter((data[:,0], data[:, indx+1])).opts(size=10, marker='o', axiswise=True, color=ROI_colors[indx]))
        
    return hv.Overlay(list_of_curves).opts(axiswise=True, ).redim.range(x=(data[0][0],data[0][-1]))
    
#    pts = list(zip(data[:,0], data[:,1]))
#    return hv.Curve(pts).opts(xlabel='scan num.', ylabel='amplitude, [au]', axiswise=True, width=600, height=450, show_grid=True, tools=['hover'], color=ROI_colors[indx])* hv.Scatter(pts).opts(size=10, marker='o', axiswise=True, color=ROI_colors[indx])

def make_image_plot(data):
    H_VMI = data.T
    return hv.Image(H_VMI, bounds=[bins_VMI[0][0], bins_VMI[1][0], bins_VMI[0][-1], bins_VMI[1][-1]],).opts(xlabel='h', ylabel='v', axiswise=True, logz=True, clim=(1e-3, None), cmap="jet", frame_height=400, frame_width=400)

def make_histo_plot(data):
    global roi_select
    roi_indxs = [roi_select.options.index(i) for i in roi_select.value]
    selected_colors = [ROI_colors[i] for i in roi_indxs]
    selected_rois = [list(Element_ROI.values())[i] for i in roi_indxs]

    mass_rois = [hv.Rectangles((i[0][0], 0, i[0][1], 1)).opts(alpha=0.3, color=c) for i, c in zip(selected_rois, selected_colors)]
    
    return hv.Curve((x_axe, data)).opts(xlabel='ToF (µs)', ylabel='normalized amplitude', axiswise=True, width=1100, show_grid=True, tools=['hover']) * hv.Overlay(mass_rois)

pn.serve(pn.Row(pn.Column(roi_select, reset_button),
                pn.Column(pn.Row(hv.DynamicMap(make_roi_scan_plot, streams=[pipe_roi_scan]),
                                 hv.DynamicMap(make_image_plot, streams=[pipe_tof])),
                          hv.DynamicMap(make_histo_plot,  streams=[pipe_histo]))),)
#         port=SERVE_PORT)



In [ ]:
def make_roi_scan_plot(data):
    global roi_select
    list_of_curves = []
    roi_indxs = [roi_select.options.index(i) for i in roi_select.value]
    for indx in roi_indxs:
        pts = list(zip(data[:,0], data[:, indx+1]))
        list_of_curves.append(hv.Curve(pts).opts(xlabel='scan num.', ylabel='amplitude, [au]', axiswise=True, framewise=True, width=600, height=450, show_grid=True, tools=['hover'], color=ROI_colors[indx])* hv.Scatter(pts).opts(size=10, marker='o', axiswise=True, framewise=True, color=ROI_colors[indx]))
        
    #return hv.Overlay(list_of_curves).opts(axiswise=True, framewise=True)
    
    pts = list(zip(data[:,0], data[:,1]))
    return hv.Curve((data[:,0], data[:,1])).opts(xlabel='scan num.', ylabel='amplitude, [au]', axiswise=True, width=600, height=450, show_grid=True, tools=['hover'], color=ROI_colors[indx])* hv.Scatter(pts).opts(size=10, marker='o', axiswise=True, color=ROI_colors[indx])
  

#test_data = np.zeros((10,5), dtype=float)
#for i in range(len(test_data[0,:])):
#    test_data[:,i] = np.arange(len(test_data)+10*i)
    
make_roi_scan_plot(np.array(roi_scan_data))